# A/B Hypothesis Testing:Ad campaign performance

The main objective of this project is to test if the ads that the advertising company runs resulted in a significant lift in brand awareness. 
This is a test run and the main objective is to validate the hypothesis algorithm you built. SmartAd ran this campaign from 3-10 July 2020.
By using classical A/B testing and machine learning,smartAds plan to result insgnificant lift to the campany.

In [ ]:
#Data exploration and visualization

#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs
import statsmodels.api as sm #
import seaborn as sns
from sklearn import preprocessing  #
from sklearn.preprocessing import scale #
from scipy.stats import spearmanr #
from sklearn.preprocessing import MinMaxScaler #
from sklearn.model_selection import train_test_split #
from sklearn.linear_model import LogisticRegression #
from sklearn.tree import DecisionTreeClassifier #
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score #
from sklearn.metrics import log_loss #
from sklearn import metrics
from matplotlib import rcParams #
from matplotlib.pyplot import pie, axis, show
from scipy.stats import norm  #

In [9]:
#loading the data 
data = pd.read_csv("/Users/bez/desktop/10xAc/week2_Adcampaign/data/AdsmartABdata.csv")
data.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [5]:
#dimension of data
data.shape

(8077, 9)

In [6]:
#users who answered yes 
data[(data.yes==1)].shape

(572, 9)

In [ ]:
#users who answered no
data[(data.no==1)].shape

In [7]:
#users who neither answered "yes" nor "no"
data[(data.yes == 0)&(data.no == 0)].shape

(6834, 9)

In [12]:
#cleaning and filtering data sets with those who answered
answered= data[(data.yes == 1)|(data.no == 1)].reset_index()

In [ ]:
# droping 
answered.drop('index', axis=1, inplace=True)
answered.shape

In [ ]:
#get unique values of each column
answered.nunique()